In [1]:
%load_ext autoreload
%autoreload 2
import os, sys
import warnings
warnings.filterwarnings('ignore') # suppress sklearn deprecation warnings for now.. 

# the below checks for whether we run dowhy and auto-causality from source
root_path = root_path = os.path.realpath('../..')
try: 
    import auto_causality
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "auto-causality"))
    
try:
    import dowhy
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "dowhy"))

In [2]:
import os, sys
import warnings
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# import auto_causality
# import dowhy

from causaldata import thornton_hiv
from auto_causality import AutoCausality
from auto_causality.datasets import synth_ihdp
from auto_causality.data_utils import preprocess_dataset
from auto_causality.scoring import ate, group_ate

In [4]:
from dowhy import datasets

# scd = datasets.linear_dataset(treatment_is_category=True, num_samples=200, )

In [5]:
from causaldata import thornton_hiv

thl = thornton_hiv.load_pandas()
df_thl = thl.data
df_thl = df_thl.dropna()
df_thl.head()

,villnum,got,distvct,tinc,any,age,hiv2004
0,1.0,1.0,2.718921,2.08032,1.0,22.0,0.0
2,1.0,1.0,2.485713,1.89120,1.0,19.0,0.0
4,1.0,1.0,1.837131,0.09456,1.0,53.0,0.0
5,1.0,1.0,2.217743,0.94560,1.0,50.0,0.0
10,1.0,0.0,2.907790,0.56736,1.0,21.0,0.0


In [9]:
df_thl.columns

Index(['villnum', 'got', 'distvct', 'tinc', 'any', 'age', 'hiv2004'], dtype='object')

In [10]:
cats = lambda y: 0 if y == 0 else round(y) + 1
df_thl["treatment"] = df_thl.apply(lambda row: cats(row.tinc)m axis=1)
df_thl.head()

KeyError: 'tinc'

In [6]:
df_thl.describe()

,villnum,got,distvct,tinc,any,age,hiv2004
count,2825.000000,2825.000000,2825.000000,2825.000000,2825.000000,2825.000000,2825.000000
mean,62.870442,0.691681,2.014541,1.005348,0.780177,33.396106,0.057699
std,49.136519,0.461881,1.268244,0.898336,0.414200,13.649935,0.252182
min,1.000000,0.000000,0.032887,0.000000,0.000000,11.000000,-1.000000
25%,12.000000,0.000000,1.031732,0.189120,1.000000,22.000000,0.000000
50%,57.000000,1.000000,1.682428,0.945600,1.000000,32.000000,0.000000
75%,111.000000,1.000000,2.795259,1.891200,1.000000,43.000000,0.000000
max,145.000000,1.000000,5.191559,2.836800,1.000000,80.000000,1.000000


In [7]:
data_df = synth_ihdp()
data_df.head()

,treatment,y_factual,x1,x2,x3,x4,x5,x6,x7,x8,...,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25
0,1,5.599916,-0.528603,-0.343455,1.128554,0.161703,-0.316603,1.295216,1,0,...,1,1,1,1,0,0,0,0,0,0
1,0,6.875856,-1.736945,-1.802002,0.383828,2.244320,-0.629189,1.295216,0,0,...,1,1,1,1,0,0,0,0,0,0
2,0,2.996273,-0.807451,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,0,0,...,1,0,1,1,0,0,0,0,0,0
3,0,1.366206,0.390083,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0,0,...,1,0,1,1,0,0,0,0,0,0
4,0,1.963538,-1.045229,-0.602710,0.011465,0.161703,0.683672,-0.360940,1,0,...,1,1,1,1,0,0,0,0,0,0
